In [ ]:
import pickle
import random
import matplotlib.pyplot as plt
import pandas as pd
import csv

# 读取pkl文件
input_file = './Bench4Merge/DJI_init/DJI_high_dhw_results.pkl'
input_file = './Bench4Merge/DJI_init/DJI_high_dhw_results.pkl'
input_file = './Bench4Merge/DJI_init/DJI_high_dhw_results.pkl'

with open(input_file, 'rb') as f:
    data = pickle.load(f)

# 随机选择一个key
random_key = random.choice(list(data.keys()))

# 获取选定key的样本数据
sample_data = data[random_key]

# 初始化绘图
plt.figure(figsize=(20, 2))

# 创建用于保存到CSV的数据列表
csv_data = []
last_frame_data = []

# 生成vehicle-ID为0的车辆数据
initial_x = 93
initial_y = 7
v_x = 6  # 5 m/s
heading = 0
v_y = 0

# 计算每辆车新的x坐标
new_x_values = {}

# 组织并添加数据到CSV列表
time_steps = [0.1 * i for i in range(len(next(iter(sample_data.values()))))]

for t, time_step in enumerate(time_steps):
    x = initial_x + v_x * time_step
    y = initial_y
    
    # 初始化编号为1的车的x值
    base_x = None
    previous_gap = 0  # 初始化前一个车的间距为0
    
    for vehicle_index, (vehicle_id, vehicle_data) in enumerate(sample_data.items(), start=1):
        point = vehicle_data[t]
        vehicle_length = point[9]
        if vehicle_index == 1:
            new_x = point[0]
            base_x = new_x
        else:
            new_x = base_x + previous_vehicle_length / 2 + previous_gap + vehicle_length / 2
            base_x = new_x
        
        previous_vehicle_length = vehicle_length
        previous_gap = point[7]  # 更新前一个车的间距
        
        new_x_values.setdefault(vehicle_index, []).append(new_x)
        if point[9] == 5:
            width = 2
        else:
            width = 2.5

        if point[8] == 2:
            char = 2
        else:
            char = random.choices([1, 2], weights=[0.6, 0.4])[0]
        
        csv_data.append([time_step, vehicle_index, new_x, point[1], point[2], point[3], point[4], point[5], point[6], char, point[9], width])
        if t == len(time_steps) - 1:
            last_frame_data.append([time_step, vehicle_index, new_x, point[1], point[2], point[3], point[4], point[5], point[6], char, point[9], width])
    
    csv_data.append([time_step, 0, x, y, heading, v_x, v_y, 0, 0, 2, 5.0, 2.0])
    if t == len(time_steps) - 1:
        last_frame_data.append([time_step, 0, x, y, heading, v_x, v_y, 0, 0, 2, 5.0, 2.0])
    
    plt.plot(x, y, 'ro')  # 绘制vehicle-ID为0的车辆位置

# 使用新的x坐标绘制其他车辆的位置
for vehicle_index, x_values in new_x_values.items():
    ego_offset_values = [point[1] for point in sample_data[f'vehicle-{vehicle_index}']]
    plt.plot(x_values, ego_offset_values, marker='o', linestyle='-', label=f'vehicle-{vehicle_index}')

# 添加图例和标签
plt.xlim(0, 170)
plt.ylim(-3, 14)
plt.xlabel('x')
plt.ylabel('y')
plt.title(f'X vs. Ego Offset for {random_key}')
plt.legend()

# 在纵坐标为1.75的位置画一条贯穿整个横坐标的实线
plt.axhline(y=1.75, color='black', linestyle='-')

# 在纵坐标为5.25的位置画指定的线条
plt.plot([0, 100], [5.25, 5.25], color='black', linestyle='-')   # 实线
plt.plot([100, 150], [5.25, 5.25], color='black', linestyle='--') # 虚线
plt.plot([150, 250], [5.25, 5.25], color='black', linestyle='-')  # 实线

# 在纵坐标为8.75的位置画实线
plt.plot([0, 130], [8.75, 8.75], color='black', linestyle='-')

# 在(130, 8.75)到(150, 5.25)的位置画实线
plt.plot([130, 150], [8.75, 5.25], color='black', linestyle='-')

# 显示图像
plt.show()

# 保存到CSV文件
output_csv = f'/data/wangzm/merge/DJI_select/DJI_init/{random_key}.csv'
output_csv_2 = f'/data/wangzm/merge/Merge-HighwayEnv-RL17/cut_in2/data/trajectory_10.csv'
output_csv_last_frame = f'/data/wangzm/merge/Merge-HighwayEnv-RL17/cut_in2/data/trajectory_1.csv'
output_csv_2 = f'/data/wangzm/merge/Bench4Merge/cut_in2/data/trajectory_10.csv'
output_csv_last_frame = f'/data/wangzm/merge/Bench4Merge/cut_in2/data/trajectory_1.csv'

df = pd.DataFrame(csv_data, columns=['time', 'vehicle-ID', 'x', 'y', 'heading', 'v_x', 'v_y', 'acc_x', 'acc_y','label', 'length', 'width'])
df_last_frame = pd.DataFrame(last_frame_data, columns=['time', 'vehicle-ID', 'x', 'y', 'heading', 'v_x', 'v_y', 'acc_x', 'acc_y','label', 'length', 'width'])

df.to_csv(output_csv, index=False)
df.to_csv(output_csv_2, index=False)
df_last_frame.to_csv(output_csv_last_frame, index=False)

print(f"Data saved to {output_csv}")
print(f"Trajectory 10 saved to {output_csv_2}")
print(f"Last frame data saved to {output_csv_last_frame}")